## Parallel Workflow with Microsoft Agent Framework and Microsoft Foundry

![parallel-workflow](./Assets/parallel_workflow.png)

In [ ]:
%pip install agent-framework==1.0.0b251209 python-dotenv azure-ai-projects==2.0.0b2

### Setting Up the Environment

In [16]:
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from agent_framework import Executor, WorkflowBuilder, WorkflowContext, WorkflowOutputEvent, handler
from agent_framework import WorkflowBuilder, WorkflowViz

load_dotenv()
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model = os.getenv("AI_FOUNDRY_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

Project Endpoint:  https://carbonopsdevai4434735199.services.ai.azure.com/api/projects/demo-proj
Model:  gpt-4.1


### Defining a Function to Create a Chat Agent

In [17]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential

# Cell 2: Define async workflow
async def create_agent(agent_name: str,
                       agent_instructions: str) -> ChatAgent:
    
    # Create async Azure credential
    credential = AzureCliCredential()

    # creating the Foundry Project Client
    project_client = AIProjectClient(
        endpoint=project_endpoint,
        credential=credential
    )

    # creating a conversation using the OpenAI Client
    openai_client = project_client.get_openai_client()
    conversation = await openai_client.conversations.create()
    conversation_id = conversation.id
    print("Conversation ID: ", conversation_id)
    
    # Initialize the Azure AI Agent Client
    chat_client = AzureAIClient(project_client=project_client,
                                conversation_id=conversation_id,
                                model_deployment_name=model)

    try:
        agent = chat_client.create_agent(
            name=agent_name,
            instructions=agent_instructions,
        )

        print("{} Agent created successfully!".format(agent_name))
        return agent

    finally:
        # Clean up async clients
        await chat_client.close()
        await credential.close()

### Creating the Location Picker Agent

In [18]:
location_picker_agent = await create_agent(
    agent_name = "Location-Picker-Agent",
    agent_instructions = """You are a helpful assistant that helps users pick a location for their vacation."""
)

Conversation ID:  conv_17d46d8d26b850dd00fcKJ9d0VsvtRFjEMjlaNuHt09HDukiyc
Location-Picker-Agent Agent created successfully!


### Creating the Destination Recommender Agent

In [19]:
destination_recommender_agent = await create_agent(
    agent_name = "Destination-Recommender-Agent",
    agent_instructions = """You are a travel expert that provides personalized vacation recommendations based on user preferences and locations. """
)

Conversation ID:  conv_9ededd616290d6390050wWkp8dvhjhnepuvprSbN622gydf1jb
Destination-Recommender-Agent Agent created successfully!


### Creating the Weather Agent

In [20]:
weather_agent = await create_agent(
    agent_name = "Weather-Agent",
    agent_instructions = """You are a weather expert that provides accurate and up-to-date weather information for various locations selected"""
)

Conversation ID:  conv_62e6ab5e6bff990000SDO9g00nPqO2hwoqchdHekYeGLYvrjUr
Weather-Agent Agent created successfully!


### Creating the Cuisine Suggestion Agent

In [21]:
cuisine_suggestion_agent = await create_agent(
    agent_name = "Cuisine-Suggestion-Agent",
    agent_instructions = """You are a culinary expert that suggests popular local cuisines and dining options based on the selected vacation destinations."""
)

Conversation ID:  conv_be1b0e850eeb877400UNeVlKHwBCF0RT3oYirJKbz4vIPy6Ez3
Cuisine-Suggestion-Agent Agent created successfully!


### Creating the Itinerary Planner Agent

In [22]:
itinerary_planner_agent = await create_agent(
    agent_name = "Itinerary-Planner-Agent",
    agent_instructions = """You are an itinerary planning expert that creates detailed travel itineraries based on user preferences, selected destinations, weather conditions, and local cuisine options."""
)

Conversation ID:  conv_cdbb59df7b842d7e00woStgh88WFsf7odGUoKdLONHvgJQaGLT
Itinerary-Planner-Agent Agent created successfully!


### Creating the Location Selector Agent Executor

In [23]:
class LocationSelectorExecutor(Executor):

    @handler
    async def handle(self, user_query: str, ctx: WorkflowContext[str]) -> None:
        response = await location_picker_agent.run(user_query)

        await ctx.send_message(str(response))

### Creating the Destination Recommender Agent Executor

In [24]:
class DestinationRecommenderExecutor(Executor):

    @handler
    async def handle(self, location: str, ctx: WorkflowContext[str]) -> None:
        response = await destination_recommender_agent.run(location)

        await ctx.send_message(str(response))

### Creating the Weather Agent Executor

In [25]:
class WeatherExecutor(Executor):

    @handler
    async def handle(self, location: str, ctx: WorkflowContext[str]) -> None:
        response = await weather_agent.run(location)

        await ctx.send_message(str(response))

### Creating the Cuisine Suggestion Agent Executor

In [26]:
class CuisineSuggestionExecutor(Executor):

    @handler
    async def handle(self, location: str, ctx: WorkflowContext[str]) -> None:
        response = await cuisine_suggestion_agent.run(location)

        await ctx.send_message(str(response))

### Creating the Itinerary Planner Agent Executor

In [27]:
class ItineraryPlannerExecutor(Executor):

    @handler
    async def handle(self, results: list[str], ctx: WorkflowContext[str]) -> None:
        response = await itinerary_planner_agent.run(results)

        await ctx.yield_output(str(response))

### Building the Parallel Workflow

In [28]:
# Create the executor instances / objects
location_selector_executor = LocationSelectorExecutor(id="LocationSelector")
destination_recommender_executor = DestinationRecommenderExecutor(id="DestinationRecommender")
weather_executor = WeatherExecutor(id="Weather")
cuisine_suggestion_executor = CuisineSuggestionExecutor(id="CuisineSuggestion")
itinerary_planner_executor = ItineraryPlannerExecutor(id="ItineraryPlanner")

# Build the workflow
workflow = (
    WorkflowBuilder()
    .set_start_executor(location_selector_executor)
    .add_fan_out_edges(location_selector_executor, [destination_recommender_executor, weather_executor, cuisine_suggestion_executor])
    .add_fan_in_edges([destination_recommender_executor, weather_executor, cuisine_suggestion_executor], itinerary_planner_executor)
    .build()
)

viz = WorkflowViz(workflow)

Adding fan-out edges with Executor or AgentProtocol instances directly is not recommended, because workflow instances created from the builder will share the same executor/agent instances. Consider using registered names for lazy initialization instead.
Adding fan-in edges with Executor or AgentProtocol instances directly is not recommended, because workflow instances created from the builder will share the same executor/agent instances. Consider using registered names for lazy initialization instead.


### Generating Mermaid Diagram for Visualization

In [ ]:
mermaid_content = viz.to_mermaid()

# printing mermaid content as markdown
from IPython.display import Markdown, display
display(Markdown(f"```mermaid\n{mermaid_content}\n```"))

### Running the Workflow and Streaming Events

In [ ]:
# Run the workflow and stream events in notebook
async def main():
    async for event in workflow.run_stream("help me plan a vacation to India with the following details: I love historical sites, prefer warm weather, and enjoy trying local foods."):
        print(f"Event: {event}")
        if isinstance(event, WorkflowOutputEvent):
            print(f"Workflow completed with result: {event.data}")

await main()